In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
def draw_yolo_bounding_box(img, x_center, y_center, width, height):
    """
    Draws a bounding box on the image using YOLO format coordinates.

    Parameters:
    - img: The image on which to draw the bounding box.
    - x_center, y_center, width, height: YOLO format coordinates (normalized).

    Returns:
    - img: The image with the bounding box drawn.
    """
    img_height, img_width = img.shape[:2]
    
    # Convert normalized coordinates to absolute coordinates
    x_center_abs = float(x_center) * img_width
    y_center_abs = float(y_center) * img_height
    width_abs = float(width) * img_width
    height_abs = float(height) * img_height
    
    # Calculate top-left and bottom-right coordinates
    x1 = int(x_center_abs - width_abs / 2)
    y1 = int(y_center_abs - height_abs / 2)
    x2 = int(x_center_abs + width_abs / 2)
    y2 = int(y_center_abs + height_abs / 2)
    
    # Draw the bounding box on the image
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    return img


def load_yolo_annotations(annotation_path):
    """
    Loads YOLO annotations from a text file.

    Parameters:
    - annotation_path: Path to the annotation file.

    Returns:
    - annotations: List of YOLO annotations.
    """
    annotations = []
    with open(annotation_path, 'r') as file:
        for line in file:
            parts = line.strip().split()
            if len(parts) == 5:
                class_id, x_center, y_center, width, height = parts
                annotations.append((class_id, x_center, y_center, width, height))

In [ ]:
# Paths to the image and annotation file
frame = 'quantex_at_home_id255237_2022_05_08_04_000450'
image_path = f'../../../ProcessedData/images/{frame}.jpg'  
annotation_path_1 = f'../../../ProcessedData/yolo/labels/train/{frame}.txt'  
annotation_path_2 = f'../../../ProcessedData/yolo/labels/val/{frame}.txt'  

if os.path.exists(annotation_path_1):
    annotation_path = annotation_path_1
elif os.path.exists(annotation_path_2):
    annotation_path = annotation_path_2
else:
    raise FileNotFoundError(f"Annotation file not found in either directory for frame: {frame}")

# Load the image
img = cv2.imread(image_path)

# Load YOLO annotations
annotations = load_yolo_annotations(annotation_path)

# Draw bounding boxes for all annotations
for annotation in annotations:
    class_id, x_center, y_center, width, height = annotation
    img = draw_yolo_bounding_box(img, x_center, y_center, width, height)

# Convert BGR image to RGB for matplotlib
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# Display the image with bounding boxes
plt.imshow(img_rgb)
plt.axis('off')  # Hide axes
plt.show()
